<a href="https://colab.research.google.com/github/ThumbRocket/Pre-Learning_AI_Model/blob/main/2_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# torch 의 유용한 함수들
import numpy as np
import torch
import torch.nn as nn  # neural network 모음. (e.g. nn.Linear, nn.Conv2d, BatchNorm, Loss functions 등등)
import torch.optim as optim  # Optimization algorithm 모음, (e.g. SGD, Adam, 등등)
import torch.nn.functional as F  # 파라미터가 필요없는 Function 모음
import torchvision.datasets as datasets  # 표준 데이터 세트 모음
import torchvision.transforms as transforms  # 데이터 세트에 적용 할 수있는 변환 관련 함수 모음
from torch.utils.data import DataLoader  # 데이터 세트 관리 및 미니 배치 생성을 위한 함수 모음
from torch.utils.data import TensorDataset # 텐서데이터셋
import torch.backends.cudnn as cudnn  # cudnn을 다루기 위한 값 모음

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Linear Regression Model

In [3]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[5], [7], [9]])

In [4]:
x_train, x_train.shape

(tensor([[1.],
         [2.],
         [3.]]),
 torch.Size([3, 1]))

In [5]:
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [6]:
W, W.shape

(tensor([0.], requires_grad=True), torch.Size([1]))

In [7]:
hypothesis = x_train * W + b

In [8]:
hypothesis

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)

In [9]:
(hypothesis - y_train) ** 2 / 3

tensor([[ 8.3333],
        [16.3333],
        [27.0000]], grad_fn=<DivBackward0>)

In [10]:
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(51.6667, grad_fn=<MeanBackward0>)


In [11]:
W

tensor([0.], requires_grad=True)

In [12]:
optimizer = optim.SGD([W, b], lr=0.01)

In [13]:
optimizer.zero_grad()
cost.backward()
optimizer.step()

### 1 epoch 실행 시 W 변화

In [14]:
W

tensor([0.3067], requires_grad=True)

In [15]:
epochs = 3000

for epoch in range(epochs+1):

    hypothesis = x_train * W + b
    cost = torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad() # RNN에서는 전파과정에 gradient add 동작 존재
    cost.backward()
    optimizer.step()

    if epoch % 200 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, epochs, W.item(), b.item(), cost.item()
            ))

Epoch    0/3000 W: 0.579, b: 0.265 Cost: 40.932430
Epoch  200/3000 W: 2.481, b: 1.908 Cost: 0.172035
Epoch  400/3000 W: 2.297, b: 2.325 Cost: 0.065691
Epoch  600/3000 W: 2.184, b: 2.583 Cost: 0.025084
Epoch  800/3000 W: 2.113, b: 2.742 Cost: 0.009578
Epoch 1000/3000 W: 2.070, b: 2.841 Cost: 0.003658
Epoch 1200/3000 W: 2.043, b: 2.902 Cost: 0.001397
Epoch 1400/3000 W: 2.027, b: 2.939 Cost: 0.000533
Epoch 1600/3000 W: 2.017, b: 2.962 Cost: 0.000204
Epoch 1800/3000 W: 2.010, b: 2.977 Cost: 0.000078
Epoch 2000/3000 W: 2.006, b: 2.986 Cost: 0.000030
Epoch 2200/3000 W: 2.004, b: 2.991 Cost: 0.000011
Epoch 2400/3000 W: 2.002, b: 2.995 Cost: 0.000004
Epoch 2600/3000 W: 2.001, b: 2.997 Cost: 0.000002
Epoch 2800/3000 W: 2.001, b: 2.998 Cost: 0.000001
Epoch 3000/3000 W: 2.001, b: 2.999 Cost: 0.000000


In [16]:
model = nn.Linear(1, 1)
list(model.parameters())

[Parameter containing:
 tensor([[-0.9184]], requires_grad=True),
 Parameter containing:
 tensor([0.3281], requires_grad=True)]

In [17]:
x_train, model(x_train)

(tensor([[1.],
         [2.],
         [3.]]),
 tensor([[-0.5903],
         [-1.5087],
         [-2.4271]], grad_fn=<AddmmBackward0>))

In [18]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [19]:
model(x_train), y_train, list(model.parameters())

(tensor([[-0.5903],
         [-1.5087],
         [-2.4271]], grad_fn=<AddmmBackward0>),
 tensor([[5.],
         [7.],
         [9.]]),
 [Parameter containing:
  tensor([[-0.9184]], requires_grad=True),
  Parameter containing:
  tensor([0.3281], requires_grad=True)])

In [20]:
epochs = 3000

for epoch in range(epochs+1):

    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train) # torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 200 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
            ))

Epoch    0/3000 Cost: 78.075951
Epoch  200/3000 Cost: 0.074100
Epoch  400/3000 Cost: 0.028295
Epoch  600/3000 Cost: 0.010804
Epoch  800/3000 Cost: 0.004126
Epoch 1000/3000 Cost: 0.001575
Epoch 1200/3000 Cost: 0.000602
Epoch 1400/3000 Cost: 0.000230
Epoch 1600/3000 Cost: 0.000088
Epoch 1800/3000 Cost: 0.000033
Epoch 2000/3000 Cost: 0.000013
Epoch 2200/3000 Cost: 0.000005
Epoch 2400/3000 Cost: 0.000002
Epoch 2600/3000 Cost: 0.000001
Epoch 2800/3000 Cost: 0.000000
Epoch 3000/3000 Cost: 0.000000


### pytorch Model 활용 Multi Linear Regression Model

In [21]:
x_train = torch.FloatTensor([[73, 80, 75], [93, 88, 93], [89, 91, 90], [96, 98, 100], [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [22]:
x_train.shape, y_train.shape

(torch.Size([5, 3]), torch.Size([5, 1]))

In [23]:
model = nn.Linear(3, 1)

In [24]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1495, -0.2733, -0.0775]], requires_grad=True),
 Parameter containing:
 tensor([0.2255], requires_grad=True)]

In [25]:
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [26]:
epochs = 6000

for epoch in range(epochs+1):

    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train) # torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 600 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
            ))

Epoch    0/6000 Cost: 35806.757812
Epoch  600/6000 Cost: 0.252116
Epoch 1200/6000 Cost: 0.248653
Epoch 1800/6000 Cost: 0.245510
Epoch 2400/6000 Cost: 0.242617
Epoch 3000/6000 Cost: 0.239929
Epoch 3600/6000 Cost: 0.237404
Epoch 4200/6000 Cost: 0.235022
Epoch 4800/6000 Cost: 0.232740
Epoch 5400/6000 Cost: 0.230558
Epoch 6000/6000 Cost: 0.228463


In [27]:
model(x_train), y_train, list(model.parameters())

(tensor([[151.3266],
         [184.7156],
         [180.5291],
         [196.5000],
         [141.7206]], grad_fn=<AddmmBackward0>),
 tensor([[152.],
         [185.],
         [180.],
         [196.],
         [142.]]),
 [Parameter containing:
  tensor([[0.9122, 0.4584, 0.6377]], requires_grad=True),
  Parameter containing:
  tensor([0.2393], requires_grad=True)])

#### Class을 이용한 Model 구현

In [28]:
class MultiLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, inputdata):
        return self.linear(inputdata)

In [29]:
model = MultiLinearRegressionModel()

In [30]:
model # model의 Layer 정보

MultiLinearRegressionModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)

In [31]:
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [32]:
epochs = 3000

for epoch in range(epochs+1):

    prediction = model.forward(x_train)
    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 600 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
            ))

Epoch    0/3000 Cost: 3500.848389
Epoch  600/3000 Cost: 1.266888
Epoch 1200/3000 Cost: 0.996098
Epoch 1800/3000 Cost: 0.799549
Epoch 2400/3000 Cost: 0.656681
Epoch 3000/3000 Cost: 0.552599


In [33]:
model(x_train), y_train, list(model.parameters())

(tensor([[151.9473],
         [184.2736],
         [180.7598],
         [196.7585],
         [140.9612]], grad_fn=<AddmmBackward0>),
 tensor([[152.],
         [185.],
         [180.],
         [196.],
         [142.]]),
 [Parameter containing:
  tensor([[0.8245, 0.5593, 0.6312]], requires_grad=True),
  Parameter containing:
  tensor([-0.3259], requires_grad=True)])

#### Pytorch Dataset 이용

In [34]:
x_train  =  torch.FloatTensor([[73,  80,  75],
                               [93,  88,  93],
                               [89,  91,  90],
                               [96,  98,  100],
                               [73,  66,  70]])
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

dataset = TensorDataset(x_train, y_train)

In [35]:
dataset, len(dataset), dataset[0], dataset[2], dataset[3]

(<torch.utils.data.dataset.TensorDataset at 0x7daa4452b490>,
 5,
 (tensor([73., 80., 75.]), tensor([152.])),
 (tensor([89., 91., 90.]), tensor([180.])),
 (tensor([ 96.,  98., 100.]), tensor([196.])))

In [36]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [37]:
dataloader, len(dataloader)

for index, batchdata in enumerate(dataloader):
    print(batchdata[0], batchdata[1])

tensor([[ 96.,  98., 100.],
        [ 89.,  91.,  90.]]) tensor([[196.],
        [180.]])
tensor([[73., 66., 70.],
        [93., 88., 93.]]) tensor([[142.],
        [185.]])
tensor([[73., 80., 75.]]) tensor([[152.]])


In [38]:
model = nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

In [39]:
epochs = 60
for epoch in range(epochs+1):
    for index, batchdata in enumerate(dataloader):

        prediction = model.forward(batchdata[0])
        cost = F.mse_loss(prediction, batchdata[1])

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(
            "Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}".format(
                epoch, epochs, index + 1, len(dataloader), cost.item()
            )
        )

Epoch    0/60 Batch 3/3 Cost: 23283.623047
Epoch   10/60 Batch 3/3 Cost: 1023.903320
Epoch   20/60 Batch 3/3 Cost: 91.202271
Epoch   30/60 Batch 3/3 Cost: 1.816506
Epoch   40/60 Batch 3/3 Cost: 1.016886
Epoch   50/60 Batch 3/3 Cost: 0.062928
Epoch   60/60 Batch 3/3 Cost: 2.386171


#### Custom Dataset 사용

In [40]:
x_train  =  np.array([[73,  80,  75],
                               [93,  88,  93],
                               [89,  91,  90],
                               [96,  98,  100],
                               [73,  66,  70]], dtype=float)
y_train  =  np.array([[152],  [185],  [180],  [196],  [142]], dtype=float)

In [41]:
x_train.shape, y_train.shape

((5, 3), (5, 1))

In [42]:
class CustomDataset(TensorDataset):
    def __init__(self, x_train, y_train):
        super().__init__()
        self.x_train = x_train
        self.y_train = y_train

    def __len__(self):
        return len(x_train)

    def __getitem__(self, index):
        return torch.FloatTensor(self.x_train[index]), torch.FloatTensor(self.y_train[index])

In [43]:
customdataset = CustomDataset(x_train, y_train)

In [44]:
customdataset, len(customdataset), customdataset[0]

(<__main__.CustomDataset at 0x7daa44568520>,
 5,
 (tensor([73., 80., 75.]), tensor([152.])))

In [45]:
dataloader = DataLoader(customdataset, batch_size=2, shuffle=True)

In [46]:
model = torch.nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [47]:
epochs = 60
for epoch in range(epochs+1):
    for index, batchdata in enumerate(dataloader):

        prediction = model.forward(batchdata[0])
        cost = F.mse_loss(prediction, batchdata[1])

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(
            "Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}".format(
                epoch, epochs, index + 1, len(dataloader), cost.item()
            )
        )

Epoch    0/60 Batch 3/3 Cost: 4052.162842
Epoch   10/60 Batch 3/3 Cost: 13.085701
Epoch   20/60 Batch 3/3 Cost: 12.968835
Epoch   30/60 Batch 3/3 Cost: 10.222410
Epoch   40/60 Batch 3/3 Cost: 19.410065
Epoch   50/60 Batch 3/3 Cost: 12.412139
Epoch   60/60 Batch 3/3 Cost: 2.883508
